In [9]:
import numpy as np
import pandas as pd
import json
# from pandas.io.json import json_normalize
import requests
from datetime import datetime, timedelta
from ast import literal_eval
import dataloader
import time
import helper
import pyodbc

In [ ]:
apikey = helper.get_apikey()

In [2]:
datetime.today()
fetchdate = datetime.strftime(datetime.today() - timedelta(days=1), '%m_%d_%Y')
fetchdate

'12_21_2022'

In [46]:
dataurl = f'https://api.themoviedb.org/3/configuration/languages?api_key={apikey}'
data = json.loads(requests.get(dataurl).text)
languages = pd.DataFrame.from_dict(data).rename(columns={'iso_639_1': 'code'})
custom = {"code": "varchar(10) PRIMARY KEY",}
dataloader.full_load_with_index(df=languages, tbl="Languages", custom=custom)

DataFrame full loaded to Table: Languages


In [3]:
cstring = helper.get_connstring()
with pyodbc.connect(cstring) as conn:
    # query = "select id from movieids"
    query = "select id from movieids where id not in (select id from moviesraw)"
    ids = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_6989/2405849386.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ids = pd.read_sql(query, conn)


In [ ]:
x=2
y=1

for chunk in np.array_split(ids['id'], 10000):
    session = requests.Session()
    df_movies = None 
    dfs=[]
    for i in chunk:
        print(f"chunk: {x}, Record: {y}") 
        dataurl = f'https://api.themoviedb.org/3/movie/{i}?api_key={apikey}'
        data = json.loads(session.get(dataurl, stream=False).text)
        df = pd.json_normalize(data)
        dfs.append(df)
        y+=1
    
    df_credits = pd.concat(dfs)

    
    
    # df_movies['vote_average'] = [format(i, '.3f') for i in df_movies['vote_average']]
    # df_movies['popularity'] = [format(i, '.3f') for i in df_movies['popularity']]
    
    # df_movies.to_csv('movies.csv', index=False)

            
    if x==1:   
        custom = {"id": "int PRIMARY KEY", "revenue": "bigint", "budget": "bigint"}
        dataloader.full_load_with_index(df=df_movies, tbl="MoviesRaw", custom=custom)
    else:
        # df_movies = pd.read_csv('movies.csv', engine='python')
        # print(df_movies.shape)
        # df_movies.dropna(subset=['id'], axis=0, inplace=True)
        # df_movies = df_movies.astype('object')
        dataloader.inc_load_with_index(df=df_movies, tbl="MoviesRaw")
    print(f"Loaded chunk: {x}")  
    x+=1
    session.close()
    del dfs
    del session
    

In [34]:
dataurl = f'https://api.themoviedb.org/3/movie/{4951}/credits?api_key={apikey}'
data = json.loads(requests.get(dataurl, stream=False).text)
df = pd.json_normalize(data)
df_cast = (
    df[['id','cast']].assign(cast = lambda x: x['cast'].apply(lambda x: literal_eval(str(x))))
      .explode('cast')
      .dropna()
      .assign(castid = lambda x: x['cast'].apply(lambda y: y['id']),
              character = lambda x: x['cast'].apply(lambda y: y['character']),
              )
      .rename(columns={'id':'movieid'})
      .drop('cast', axis=1)
    )

df_crew = (
    df[['id','crew']].assign(crew = lambda x: x['crew'].apply(lambda x: literal_eval(str(x))))
      .explode('crew')
      .dropna()
      .assign(crewid = lambda x: x['crew'].apply(lambda y: y['id']),
              department = lambda x: x['crew'].apply(lambda y: y['department']),
              )
      .rename(columns={'id':'movieid'})
      .drop('crew', axis=1)
    )

In [38]:
df_cast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 0 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movieid    44 non-null     int64 
 1   castid     44 non-null     int64 
 2   character  44 non-null     object
dtypes: int64(2), object(1)
memory usage: 1.4+ KB


In [39]:
df_crew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 0
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movieid     81 non-null     int64 
 1   crewid      81 non-null     int64 
 2   department  81 non-null     object
dtypes: int64(2), object(1)
memory usage: 2.5+ KB
